In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

# open reviews as a df

In [2]:
df = pd.read_csv('data/clorox_data.csv')
df

,brand,product_title,proxy_date,retailer,category,subcategory,review_text,star_rating,topic,brand_type
0,Clorox,Clorox Wring Clean Cotton Mop,2024-07-08,Target,CLEANING,FLOOR CLEANERS,"The mop is cheap, the wringing attachment does...",1.0,Product review: Click n Clean multi-surface sp...,Clorox
1,Rapid,"32 oz. Rapid Clean Remediation, Trigger Spray ...",2024-07-08,Home Depot,CLEANING,BATHROOM CLEANERS MILDEW CLEANERS,This product was recommended for a front load ...,1.0,Mold removal and prevention products,Competitor
2,Mrs. Meyer's Clean Day,Mrs Meyers Clean Day Cleaner Plastic Bottle Mu...,2024-07-08,Target,CLEANING,SPRAY CLEANERS ALL PURPOSE CLEANERS,"Smells so good, definitely my favorite scent",5.0,Fall scents and their popularity,Competitor
3,Pourri,Poo Pourri Plastic Spray Bottle Toilet Oil Fre...,2024-07-08,Target,CLEANING,ODOR CONTROLLING AIR FRESHENERS,"I keep multiple bottles in the bathroom, and c...",5.0,Household cleaning and sanitization tips,Competitor
4,Twist & Shout,Twist & Shout Spin Mop & Bucket System with 1 ...,2024-07-01,Costco,CLEANING,FLOOR CLEANERS,I recently just got this mop love how easy it ...,5.0,Product review: Click n Clean multi-surface sp...,Competitor
...,...,...,...,...,...,...,...,...,...,...
670793,Neutrogena,Neutrogena Liquid Gel Cleansing Fragrance Free...,2022-07-04,Kroger,PERSONAL CARE,FACE CARE FACIAL CLEANSERS,I don't know what in this could possibly do th...,1.0,Positive reviews for a makeup remover (undeter...,Competitor
670794,Hero Cosmetics,Hero Lightning Wand Acne Treatment Serum Rp 1 ...,2022-07-04,Kroger,PERSONAL CARE,FACE CARE ACNE TREATMENTS,I’ve used this product for about 1month and it...,5.0,Dark spot fading progress with skin products,Competitor
670795,Cetaphil,CETAPHIL LIQUID GEL CLEANSING AO SCENT 1 CT 16...,2022-07-04,Kroger,PERSONAL CARE,FACE CARE FACIAL CLEANSERS,Love this cleanser for myself and my daughter....,5.0,Positive reviews for a makeup remover (undeter...,Competitor
670796,Neutrogena,Neutrogena Liquid Gel Cleansing Fragrance Free...,2022-07-04,Kroger,PERSONAL CARE,FACE CARE FACIAL CLEANSERS,I don't know what in this could possibly do th...,1.0,Positive reviews for a makeup remover (undeter...,Competitor


In [5]:
processed  = pd.read_csv('../data/processed_reviews.csv')

In [15]:
i = 343
print(df.loc[i, 'review_text'])
print(processed.loc[i, 'review_text']) 

Love the smell but just 1 doesn't last a full 30 days . I have 4 and I have to keep changing them out every couple of weeks . This why I have 4 stars
love the smell but just 1 doesnt last a full 30 day i have 4 and i have to keep change them out every couple of week this why i have 4 star


# preprocess reviews

In [9]:
import string
import nltk
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    # Tokenize the text
    words = nltk.word_tokenize(text)
    
    # Perform part-of-speech tagging
    pos_tags = nltk.pos_tag(words)
    
    # Lemmatize words based on their part of speech
    lemmatized_words = []
    for word, pos in pos_tags:
        if pos.startswith('N'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='n'))
        elif pos.startswith('V'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='v'))
        elif pos.startswith('R'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='r'))
        elif pos.startswith('J'):
            lemmatized_words.append(lemmatizer.lemmatize(word, pos='a'))
        else:
            lemmatized_words.append(word)
    
    return ' '.join(lemmatized_words)

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation + '’')
    return text.translate(translator)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jessicaluo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jessicaluo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jessicaluo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/jessicaluo/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [5]:
# copy og review_text to new column
df.loc[:, 'og_review_text'] = df['review_text'].copy()
df.head()

,brand,product_title,proxy_date,retailer,category,subcategory,review_text,star_rating,topic,brand_type,og_review_text
0,Clorox,Clorox Wring Clean Cotton Mop,2024-07-08,Target,CLEANING,FLOOR CLEANERS,"The mop is cheap, the wringing attachment does...",1.0,Product review: Click n Clean multi-surface sp...,Clorox,"The mop is cheap, the wringing attachment does..."
1,Rapid,"32 oz. Rapid Clean Remediation, Trigger Spray ...",2024-07-08,Home Depot,CLEANING,BATHROOM CLEANERS MILDEW CLEANERS,This product was recommended for a front load ...,1.0,Mold removal and prevention products,Competitor,This product was recommended for a front load ...
2,Mrs. Meyer's Clean Day,Mrs Meyers Clean Day Cleaner Plastic Bottle Mu...,2024-07-08,Target,CLEANING,SPRAY CLEANERS ALL PURPOSE CLEANERS,"Smells so good, definitely my favorite scent",5.0,Fall scents and their popularity,Competitor,"Smells so good, definitely my favorite scent"
3,Pourri,Poo Pourri Plastic Spray Bottle Toilet Oil Fre...,2024-07-08,Target,CLEANING,ODOR CONTROLLING AIR FRESHENERS,"I keep multiple bottles in the bathroom, and c...",5.0,Household cleaning and sanitization tips,Competitor,"I keep multiple bottles in the bathroom, and c..."
4,Twist & Shout,Twist & Shout Spin Mop & Bucket System with 1 ...,2024-07-01,Costco,CLEANING,FLOOR CLEANERS,I recently just got this mop love how easy it ...,5.0,Product review: Click n Clean multi-surface sp...,Competitor,I recently just got this mop love how easy it ...


In [10]:
# preprocess reviews in df

df['review_text'] = df['review_text'].apply(lambda x: x.lower())
df['review_text'] = df['review_text'].apply(lambda x: remove_punctuation(x))
df['review_text'] = df['review_text'].apply(lambda x: lemmatize_text(x))
df['review_text'].size

670798

In [12]:
print(df.shape)
df.head()

(670798, 10)


,brand,product_title,proxy_date,retailer,category,subcategory,review_text,star_rating,topic,brand_type
0,Clorox,Clorox Wring Clean Cotton Mop,2024-07-08,Target,CLEANING,FLOOR CLEANERS,the mop be cheap the wringing attachment doesn...,1.0,Product review: Click n Clean multi-surface sp...,Clorox
1,Rapid,"32 oz. Rapid Clean Remediation, Trigger Spray ...",2024-07-08,Home Depot,CLEANING,BATHROOM CLEANERS MILDEW CLEANERS,this product be recommend for a front load was...,1.0,Mold removal and prevention products,Competitor
2,Mrs. Meyer's Clean Day,Mrs Meyers Clean Day Cleaner Plastic Bottle Mu...,2024-07-08,Target,CLEANING,SPRAY CLEANERS ALL PURPOSE CLEANERS,smell so good definitely my favorite scent,5.0,Fall scents and their popularity,Competitor
3,Pourri,Poo Pourri Plastic Spray Bottle Toilet Oil Fre...,2024-07-08,Target,CLEANING,ODOR CONTROLLING AIR FRESHENERS,i keep multiple bottle in the bathroom and car...,5.0,Household cleaning and sanitization tips,Competitor
4,Twist & Shout,Twist & Shout Spin Mop & Bucket System with 1 ...,2024-07-01,Costco,CLEANING,FLOOR CLEANERS,i recently just get this mop love how easy it ...,5.0,Product review: Click n Clean multi-surface sp...,Competitor


In [11]:
# remove duplicate reviews
df_drop_dupe = df.drop_duplicates(subset='review_text')
df_drop_dupe.shape

(640606, 11)

In [12]:
# save preprocessed data to csv
df_drop_dupe.to_csv('data/processed_reviews.csv', index=False)

In [15]:
# group reviews by subcategory and convert to dictionary
grouped = df.groupby('subcategory')['review_text'].apply(list).to_dict()
grouped.keys()

dict_keys(['ABRASIVE CLEANERS', 'AIR FRESHENER', 'BATHROOM CLEANERS', 'BATHROOM CLEANERS DAILY SHOWER CLEANERS', 'BATHROOM CLEANERS GENERAL BATHROOM CLEANERS', 'BATHROOM CLEANERS LIMESCALE/HARDWATER CLEANERS', 'BATHROOM CLEANERS MILDEW CLEANERS', 'BODY CARE', 'BODY CARE BAR SOAP', 'BODY CARE BATH SOAP', 'BODY CARE BODY LOTION', 'BODY CARE BODY OIL', 'BODY CARE BODY TOOLS', 'BODY CARE BODY WASH', 'BODY CARE BODY WIPES', 'BODY CARE DEODORANT', 'BODY CARE FOOT CARE', 'BODY CARE HAIR REMOVAL', 'BODY CARE HAND CARE', 'CONSUMABLE TOOLS', 'CONSUMABLE TOOLS CLEANING CLOTHS', 'CONSUMABLE TOOLS CONSUMABLE SCRUBBERS', 'CONSUMABLE TOOLS SOAP PADS/STEEL WOOL', 'CONSUMABLE TOOLS SPONGES', 'CORE GIFTS', 'CORE GIFTS EVERYDAY KITS', 'CORE GIFTS HOLIDAY KITS', 'DILUTABLES', 'DILUTABLES NATURAL/CONCENTRATED', 'DILUTABLES PINE/DISINFECTING DILUTABLES', 'DILUTABLES SCENTED/NON-DISINFECTING DILUTABLES', 'DISH CARE', 'DISH CARE LIQUID DISH DETERGENT', 'DRAIN CARE', 'FACE CARE', 'FACE CARE ACNE TREATMENTS', '

# LSA

In [23]:
# create lsa pipeline
def create_components(n_topics):
    vectorizer = TfidfVectorizer(stop_words='english', 
                                 use_idf=True, 
                                 ngram_range=(1, 2),
                                 smooth_idf=True)
    svd_model = TruncatedSVD(n_components=n_topics,        
                             algorithm='randomized',
                             n_iter=20)
    svd_transformer = Pipeline([('tfidf', vectorizer), 
                                ('svd', svd_model)])
    return svd_transformer, vectorizer, svd_model

In [16]:
# dictionary of first 2 subcategories
groups = {k: grouped[k] for k in list(grouped)[:2]}

In [37]:
# for each subcategory, fit the pipeline to the reviews and save results to lsa_df with columns: subcategory, topic_number, top_words, sample_reviews
lsa_df = pd.DataFrame(columns=['subcategory', 'topic_number', 'top_words', 'sample_reviews'])

for subcategory, reviews in grouped.items():
    n_topics = int(min(15, len(reviews)))
    # if n_topics >= 2:
    svd_transformer, vectorizer, svd_model = create_components(n_topics)
    svd_matrix = svd_transformer.fit_transform(reviews)
    terms = vectorizer.get_feature_names_out()

    for i, topic in enumerate(svd_model.components_):
        top_words = [terms[j] for j in topic.argsort()[:-10 - 1:-1]]
        n_samples = min(5, len(reviews))
        # get representative 3 reviews for each topic
        rep_reviews = reviews.iloc[np.argsort(svd_matrix[:,i])[::-1][:3]].values
            
        new_row = pd.DataFrame({'subcategory': [subcategory], 'topic_number': [i], 'top_words': [top_words], 'sample_reviews': [rep_reviews]})
        lsa_df = pd.concat([lsa_df, new_row], ignore_index=True)

lsa_df

/Users/jessicaluo/Desktop/scrap/text_processes/lib/python3.12/site-packages/sklearn/decomposition/_truncated_svd.py:275: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var


,subcategory,topic_number,top_words,sample_reviews
0,ABRASIVE CLEANERS,0,"[clean, great, use, product, work, sink, work ...",[this product be absolutely wonderful it work ...
1,ABRASIVE CLEANERS,1,"[use, year, use year, comet, product, ive, ive...",[i like the round its a special cleaner my fir...
2,ABRASIVE CLEANERS,2,"[great, work, work great, great product, produ...","[not as good as comet, appreciate shipping odd..."
3,ABRASIVE CLEANERS,3,"[clean, bathroom, clean bathroom, kitchen, sme...",[perfect clean easy to scrub and get out stain...
4,ABRASIVE CLEANERS,4,"[good, product, good product, price, good pric...",[best product for clean stainless cookware sin...
...,...,...,...,...
1372,WOOD/FURNITURE/DUST WOOD/FURNITURE CLEANER,10,"[clean, product, great product, surface, clean...","[brings wood bake to new look, work great plea..."
1373,WOOD/FURNITURE/DUST WOOD/FURNITURE CLEANER,11,"[make, like, look, furniture, shine, make floo...","[love this item will reorder, dont have the tr..."
1374,WOOD/FURNITURE/DUST WOOD/FURNITURE CLEANER,12,"[cleaner, hardwood, hardwood floor, best, furn...",[get a lot of would get wood floor you need th...
1375,WOOD/FURNITURE/DUST WOOD/FURNITURE CLEANER,13,"[furniture, clean, leave, wood furniture, leat...",[my go to cleaner love the smell and the way i...


In [38]:
# save lsa_df to csv
lsa_df.to_csv('outputs/lsa_1-2gram_results.csv', index=False)

# Cluster similar topics

In [39]:
# remove space between 2 gram words in top_words so that when vectorized, they are treated as one entity
lsa_df['top_words'] = lsa_df['top_words'].apply(lambda x: [word.replace(' ', '') for word in x])

# create dict of subcategories and array of top words joined to one string for each topic
lsa_dict = {}
for subcategory in lsa_df['subcategory'].unique():
    sub_df = lsa_df[lsa_df['subcategory'] == subcategory]
    sub_dict = {}
    for topic in sub_df['topic_number']:
        topic_df = sub_df[sub_df['topic_number'] == topic]
        sub_dict[topic] = ' '.join(topic_df['top_words'].values[0])
    lsa_dict[subcategory] = sub_dict
lsa_dict['ABRASIVE CLEANERS']

{0: 'clean great use product work sink workgreat year good love',
 1: 'use year useyear comet product ive iveuse love usecomet useproduct',
 2: 'great work workgreat greatproduct product price productwork greatprice greatclean workgood',
 3: 'clean bathroom cleanbathroom kitchen smell comet good tub toilet greatclean',
 4: 'good product goodproduct price goodprice greatproduct productclean loveproduct job excellent',
 5: 'good work comet cleaner best cleanser stain workgood goodproduct like',
 6: 'comet stainless good stainlesssteel usecomet steel like smell clean workgreat',
 7: 'best cleaner great bestcleaner price cleaning bathroom comet kitchen bestproduct',
 8: 'comet great bar keeper love barkeeper friend keeperfriend price job',
 9: 'love product best work comet loveproduct productwork pot bestproduct pan',
 10: 'clean cleaner best pot pan potpan useyear year bestcleaner ive',
 11: 'stain comet sink water hard job hardwater great remove waterstain',
 12: 'love cleaner loveproduc

In [40]:
# cluster top words representations for each subcategory
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

vectorizer = TfidfVectorizer()
n_clusters = 10

# kmeans applied to a subcategory, return cluster labels
def apply_kmeans(topics_dict, vectorizer, n_clusters):
    if len(topics_dict) > n_clusters:
        topics_vectorized = vectorizer.fit_transform(topics_dict.values())
        kmeans_model = KMeans(n_clusters=n_clusters, random_state=12)
        cluster_labels = kmeans_model.fit_predict(topics_vectorized)
        return cluster_labels
    return None

# merge topics in subcategory with the same cluster labels
def merge_topics(topics_dict, cluster_labels):
    merged_topics = {}
    for i in range(len(cluster_labels)):
        if cluster_labels[i] not in merged_topics:
            merged_topics[cluster_labels[i]] = topics_dict[i]
        else:
            merged_topics[cluster_labels[i]] += ' ' + topics_dict[i]
    return merged_topics

# remove duplicate words in merged topics
def remove_duplicates(merged_topics):
    for label, words in merged_topics.items():
        merged_topics[label] = ' '.join(list(set(words.split())))

In [41]:
# apply kmeans to each subcategory and merge topics with the same cluster labels

clustered_dict = {}
for subcategory, topics_dict in lsa_dict.items():
    cluster_labels = apply_kmeans(topics_dict, vectorizer, n_clusters)
    if cluster_labels is not None:
        merged_topics = merge_topics(topics_dict, cluster_labels)
        remove_duplicates(merged_topics)
        clustered_dict[subcategory] = merged_topics
    else:
        clustered_dict[subcategory] = topics_dict

clustered_dict['ABRASIVE CLEANERS']

{np.int32(1): 'great clean usecomet love product useyear iveuse good workgreat use useproduct comet sink year work ive',
 np.int32(0): 'great goodproduct productclean product workgood greatprice good excellent greatclean workgreat loveproduct job price work productwork goodprice greatproduct',
 np.int32(6): 'clean bathroom good tub greatclean cleanbathroom smell comet toilet kitchen',
 np.int32(7): 'best goodproduct workgood good like cleanser comet cleaner work stain',
 np.int32(8): 'stainless usecomet clean good like workgreat stainlesssteel comet smell steel',
 np.int32(4): 'best great love bathroom bar keeper barkeeper keeperfriend bestcleaner comet cleaner price job friend bestproduct cleaning kitchen',
 np.int32(5): 'best pan clean love product useyear loveproduct comet pot productwork work cleaner year bestcleaner ive potpan bestproduct',
 np.int32(2): 'great waterstain potpan kitchen bathroom hard hardwater water job comet sink pot remove pan stain',
 np.int32(9): 'loveclean st

In [42]:
# save clustered_dict to csv
clustered_df = pd.DataFrame(columns=['subcategory', 'topic_number', 'top_words'])
for subcategory, topics_dict in clustered_dict.items():
    for topic, words in topics_dict.items():
        new_row = pd.DataFrame({'subcategory': [subcategory], 'topic_number': [topic], 'top_words': [words]})
        clustered_df = pd.concat([clustered_df, new_row], ignore_index=True)
clustered_df.to_csv('outputs/lsa_1-2gram_clustered_results.csv', index=False)

In [44]:
lsa_dict['BODY CARE HAIR REMOVAL']

{0: 'box worksmall work smallcome small drybox dry comedry come'}